In [ ]:
pip install -q google-generativeai pydantic

In [ ]:
import google.colab.userdata as userdata
import google.generativeai as genai
from pydantic import BaseModel, Field

# Pydantic models for tool arguments.
class say_hi_validator(BaseModel):
    """Arguments for the 'say_hi' tool."""
    query: str = Field(description="The user's greeting message.")

class say_bye_validator(BaseModel):
    """Arguments for the 'say_bye' tool."""
    query: str = Field(..., description="The user's farewell message.")

# Your Python tools as regular functions.
def say_hi(query: str) -> str:
    """
    Use this tool when the user greets you, says hello, or starts a conversation.
    """
    return "Hi!"

def say_bye(query: str) -> str:
    """
    Use this tool when the user wants to end the conversation or says goodbye.
    """
    return "Bye!"

In [ ]:
# Configure with your API key
api_key = userdata.get("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("Please set the GOOGLE_API_KEY environment variable.")
genai.configure(api_key=api_key)

# The SDK automatically uses Pydantic models to create the tool schema.
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    tools=[say_hi, say_bye]
)

In [ ]:
# System instruction (prompting) to guide the model's behavior.
system_instruction = ('''
You are an assistant designed to use tools to respond to user requests.
Your primary task is to carefully analyze the user's input and determine which of the available tools is most appropriate to accomplish their goal.

If a tool is a perfect match, call it with the correct parameters.
If no tool is suitable for the user's request, do not aexttempt to call any tool. Instead, respond directly with the exact phrase: 'You can do better!'
''')


# The conversational loop
print("Agent is ready. Type 'exit' to end the conversation.")
for turn in range(10): # A for loop to limit the number of turns for this example
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        print("Agent: Goodbye!")
        break

   # Send the user's message to the model with the system instruction
    response = model.generate_content(
        [system_instruction, user_input]
        )

    # Check if the model's response contains a tool call
    try:
        tool_call = response.candidates[0].content.parts[0].function_call
        tool_name = tool_call.name
        tool_args = dict(tool_call.args)

        print(f"Agent chose to call tool: {tool_name} with arguments: {tool_args}")

        # Execute the tool using the mapped function and the validated arguments

        tool_result = tools_map[tool_name](**tool_args)

        # Print the tool's output to the user
        print(f"Agent: {tool_result}")

    except :
        # This block runs if no tool was called. The model responded with text
        print(f"Agent: {response.text}")


Agent is ready. Type 'exit' to end the conversation.
You: hi
Agent chose to call tool: say_hi with arguments: {'query': 'hi'}
Agent: Hi!
You: bye
Agent chose to call tool: say_bye with arguments: {'query': 'bye'}
Agent: Bye!
You: fgchvjbnm
Agent: You can do better!

You: exit
Agent: Goodbye!
